<a href="https://colab.research.google.com/github/DhaniAAA/Scrapping-X/blob/main/Scrapping%20X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from webdriver_manager.chrome import ChromeDriverManager

# --- KONFIGURASI ---
# 1. SALIN DAN TEMPEL NILAI `auth_token` ANDA DI BAWAH INI.
# JANGAN BAGIKAN SCRIPT INI DENGAN COOKIE ANDA KARENA BERSIFAT RAHASIA.
AUTH_TOKEN_COOKIE = "Ganti dengan Cookie kalian"

# 2. Ubah kata kunci pencarian sesuai kebutuhan Anda
SEARCH_QUERY = "anies baswedan"

# 3. Jumlah scroll ke bawah untuk memuat lebih banyak tweet
SCROLL_COUNT = 5

# Waktu tunggu (dalam detik) antara setiap scroll agar halaman sempat memuat
SCROLL_PAUSE_TIME = 3
# --------------------


def setup_driver():
    """Menyiapkan instance WebDriver untuk Chrome."""
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        return driver
    except (WebDriverException, ValueError) as e:
        print("Error saat menyiapkan WebDriver.")
        print("Jika di Google Colab, pastikan sel instalasi Chrome sudah dijalankan.")
        print(f"Detail error: {e}")
        return None

def scroll_down(driver, scroll_count):
    """Melakukan scroll ke bawah pada halaman untuk memuat konten dinamis."""
    print(f"Melakukan {scroll_count} kali scroll ke bawah...")
    last_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        print(f"  Scroll ke-{i+1}/{scroll_count}, menunggu {SCROLL_PAUSE_TIME} detik...")
        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("  Telah mencapai akhir halaman, scroll dihentikan.")
            break
        last_height = new_height


def scrape_tweets(driver, query):
    """Mengekstrak data tweet dari halaman yang sudah dimuat."""

    # --- PERBAIKAN: LOGIN MENGGUNAKAN COOKIE ---
    # 1. Kunjungi domain dasar terlebih dahulu untuk mengatur konteks cookie
    print("Mengunjungi x.com untuk menyuntikkan cookie login...")
    driver.get("https://x.com")
    time.sleep(2) # Beri waktu sejenak

    # 2. Tambahkan cookie auth_token ke sesi browser
    if AUTH_TOKEN_COOKIE and AUTH_TOKEN_COOKIE != "GANTI_DENGAN_NILAI_AUTH_TOKEN_ANDA":
        cookie = {
            'name': 'auth_token',
            'value': AUTH_TOKEN_COOKIE,
            'domain': '.x.com',
            'path': '/',
            'secure': True,
            'httpOnly': True
        }
        driver.add_cookie(cookie)
        print("Cookie berhasil disuntikkan.")
    else:
        print("PERINGATAN: Cookie tidak diatur. Script mungkin akan terhadang halaman login.")
    # ---------------------------------------------

    search_url = f"https://x.com/search?q={query}&src=typed_query&f=live"
    print(f"Mengunjungi halaman pencarian: {search_url}")
    driver.get(search_url)

    try:
        print("Menunggu konten tweet muncul (maksimal 20 detik)...")
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//article[@data-testid='tweet']"))
        )
        print("Konten tweet terdeteksi. Melanjutkan proses.")
    except TimeoutException:
        print("Batas waktu menunggu habis. Tidak ada tweet yang ditemukan.")
        print("Penyebab umum: Nilai cookie salah/kadaluarsa atau perubahan dari pihak X.")
        driver.save_screenshot('debug_screenshot.png')
        print("Screenshot 'debug_screenshot.png' telah disimpan untuk diperiksa.")
        return []

    scroll_down(driver, SCROLL_COUNT)

    print("Mulai mengambil data tweet...")
    tweets_data = []

    tweet_articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
    print(f"Ditemukan {len(tweet_articles)} tweet.")

    for tweet in tweet_articles:
        try:
            username = tweet.find_element(By.XPATH, ".//div[@data-testid='User-Name']//span").text
            handle = tweet.find_element(By.XPATH, ".//span[contains(text(), '@')]").text
            timestamp = tweet.find_element(By.XPATH, ".//time").get_attribute('datetime')
            tweet_text = tweet.find_element(By.XPATH, ".//div[@data-testid='tweetText']").text.replace('\n', ' ')

            reply_count = tweet.find_element(By.XPATH, ".//button[@data-testid='reply']").text
            retweet_count = tweet.find_element(By.XPATH, ".//button[@data-testid='retweet']").text
            like_count = tweet.find_element(By.XPATH, ".//button[@data-testid='like']").text

            tweet_url_elements = tweet.find_elements(By.XPATH, ".//a[contains(@href, '/status/')]")
            tweet_url = tweet_url_elements[0].get_attribute('href') if tweet_url_elements else 'N/A'

            tweets_data.append({
                "username": username,
                "handle": handle,
                "timestamp": timestamp,
                "tweet_text": tweet_text,
                "url": tweet_url,
                "reply_count": reply_count or "0",
                "retweet_count": retweet_count or "0",
                "like_count": like_count or "0",
            })
        except Exception:
            continue

    return tweets_data

def main():
    """Fungsi utama untuk menjalankan proses scraping."""
    if not AUTH_TOKEN_COOKIE or AUTH_TOKEN_COOKIE == "GANTI_DENGAN_NILAI_AUTH_TOKEN_ANDA":
        print("Error: Harap isi variabel AUTH_TOKEN_COOKIE dengan nilai cookie Anda.")
        return

    driver = setup_driver()
    if not driver:
        return

    try:
        scraped_data = scrape_tweets(driver, SEARCH_QUERY)

        if not scraped_data:
            print("Tidak ada data yang berhasil diambil. Coba periksa 'debug_screenshot.png' jika file itu ada.")
            return

        df = pd.DataFrame(scraped_data)
        output_filename = "tweets.csv"
        df.to_csv(output_filename, index=False, encoding='utf-8')

        print(f"\nProses selesai. Data telah disimpan di {output_filename}")
        print(f"Total tweet yang berhasil diambil: {len(df)}")
        print(df.head())

    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    main()


Mengunjungi x.com untuk menyuntikkan cookie login...
Cookie berhasil disuntikkan.
Mengunjungi halaman pencarian: https://x.com/search?q=anies baswedan&src=typed_query&f=live
Menunggu konten tweet muncul (maksimal 20 detik)...
Konten tweet terdeteksi. Melanjutkan proses.
Melakukan 5 kali scroll ke bawah...
  Scroll ke-1/5, menunggu 3 detik...
  Scroll ke-2/5, menunggu 3 detik...
  Scroll ke-3/5, menunggu 3 detik...
  Scroll ke-4/5, menunggu 3 detik...
  Scroll ke-5/5, menunggu 3 detik...
Mulai mengambil data tweet...
Ditemukan 4 tweet.

Proses selesai. Data telah disimpan di tweets.csv
Total tweet yang berhasil diambil: 4
         username            handle                 timestamp  \
0        Raihanna  @Raihann83638626  2025-06-09T01:40:32.000Z   
1            Grok             @grok  2025-06-09T01:36:41.000Z   
2            Grok             @grok  2025-06-09T01:24:35.000Z   
3  Dede Budhyarto       @kangdede78  2025-06-09T01:19:07.000Z   

                                          twe

In [17]:
df = pd.read_csv("/content/tweets.csv")
df

,username,handle,timestamp,tweet_text,url,reply_count,retweet_count,like_count
0,Raihanna,@Raihann83638626,2025-06-09T01:40:32.000Z,"Nih abah aing mah ga norax, ga pansos, ga riya...",https://x.com/Raihann83638626/status/193188911...,0,0,0
1,Grok,@grok,2025-06-09T01:36:41.000Z,@KenapeAje @nuphogosong_ @Heraloebss Ormas ...,https://x.com/grok/status/1931888148432466029,1,0,0
2,Grok,@grok,2025-06-09T01:24:35.000Z,"""Ormas"" kemungkinan merujuk pada Ormas Gerakan...",https://x.com/grok/status/1931885099647377860,1,0,0
3,Dede Budhyarto,@kangdede78,2025-06-09T01:19:07.000Z,"Stlh GAGAL TOTAL dgn isu lingkungan, ternyta s...",https://x.com/kangdede78/status/19318837238569...,47,35,91
